In [1]:
import json
import pandas as pd
import numpy as np
import os
from pprint import pprint
import gensim
from sklearn import svm

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
with open(r'C:\Users\inmmj\Desktop\4학년1학기\기계학습\RnE\RnE\data\reviews_Digital_Music.json') as f_r:
    for line in f_r.readlines()[:10]:
        each = json.loads(line)
        rating = each['overall']
        pprint(each)
word2vec_model = gensim.models.Word2Vec.load(r'C:\Users\inmmj\Desktop\4학년1학기\기계학습\RnE\RnE\Word2vec\wiki_en_model')

rating_5_count = 0
rating_1_count = 0

train_list = []

with open(r'C:\Users\inmmj\Desktop\4학년1학기\기계학습\RnE\RnE\data\reviews_Digital_Music.json', 'r') as f_r:
    for line in f_r.readlines():
        each = json.loads(line)
        rating = each['overall']
        
        if rating == 5.0 and rating_5_count < 20000:
            rating_5_count += 1
            each['class_label'] = 1
            train_list.append(each)
        elif rating == 1.0 and rating_1_count < 20000:
            rating_1_count += 1
            each['class_label'] = 0
            train_list.append(each)

{'asin': '5555991584',
 'helpful': [4, 5],
 'overall': 5.0,
 'reviewText': 'The anthemic title track begins &quot;The Memory Of '
               'Trees&quot;, her fourth CD release.  Wordless vocalizations '
               'and forceful percussion propel this song into the '
               'stratosphere.  &quot;Anywhere Is&quot;, the lead single from '
               'this CD, is typical of her earlier fare like &quot;Orinoco '
               'Flow&quot;, but contains a magic all its own.  The most '
               'endearing aspect of this song is its lyrics.  Roma Ryan, '
               "Enya's long time lyricist, deserves much credit for her "
               'amazing lyrics on this song.  Relax and fall into the images '
               'the music and the words make together.  &quot;Pax Deorum&quot; '
               'is like a hurricane to me.  It begins ominously, like the calm '
               'before the storm.  It builds into a frighteningly wicked storm '
               'of imme

In [3]:
%%time
df = pd.DataFrame(train_list)

train_X = []
train_Y = []

for class_label, reviews in df.groupby('class_label'):
    for each in reviews.iterrows():
        review = each[1]['reviewText'].lower()
        words = review.split(' ')
        svec = np.zeros(word2vec_model.vector_size)
        count = 0
        
        for word in words:
            if word in word2vec_model.wv.vocab:
                svec += word2vec_model[word]
                count += 1
        if count > 0:
            svec /= count
            
        train_X.append(svec)
        train_Y.append(class_label)
print(len(train_Y))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


40000
Wall time: 1min 9s


In [4]:
len(train_X)


40000

In [5]:
clf = svm.SVC(kernel='linear')
clf.fit(train_X, train_Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [6]:
def test(query_sentence):
    query_words = query_sentence.split(' ')
    svec = np.zeros(word2vec_model.vector_size)
    count = 0
    for query_word in query_words:
        if query_word in word2vec_model.wv.vocab:
            svec += word2vec_model[query_word]
            count += 1

    if count > 0:
        svec = svec/count
    
    result = clf.predict([svec])
    return result

In [7]:
predict_result = test("I love the sound")

if predict_result:
    print('긍정')
else:
    print('부정')

긍정


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
